In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from matplotlib.colors import LogNorm

In [33]:
def load_image(path: str):
    """
    Load in FITS image from a path as a numpy array.
    """
    data = fits.getdata(path)
    return data

def view_fits(data, vmin: int = 1, vmax: int = 65000, log: bool = True):
    """
    View a FITS image using its numpy array.
    Use vmin and vmax to scale the image.
    Use log to swap between a linear and a logarithmic scale.
    """
    plt.figure()
    
    if log:
        plt.imshow(data, origin = 'lower', norm = LogNorm(vmin = vmin, vmax = vmax), cmap = 'inferno')
    else:
        plt.imshow(data, origin = 'lower', vmin = vmin, vmax = vmax, cmap = 'inferno')
        
    cb = plt.colorbar()
    cb.set_label('Signal [ADU]')
    plt.xlabel('X [px]')
    plt.ylabel('Y [px]')

In [141]:
def view_boxes_on_image(data, coords_list, side_length: int = 10, vmin: int = 1, vmax: int = 65000, log: bool = True):
    """
    View the boxes superimposed on the FITS data.
    The coords need to be in a list of the format (even if only one set of coords):
    [[x1, y1], [x2, y2], [x3, y3], ...]
    
    Set the side length of the box in pixels using side_length.
    
    Use vmin and vmax to scale the image.
    Use log to swap between a linear and a logarithmic scale.
    
    Note: If the box isn't right, try swapping the x and y coords.
    """

    def draw_box(x_coord: int, y_coord: int, side_length: int, box_number: int):
        """
        Draws a box.
        """
        l = side_length//2
        
        plt.plot([x_coord - l, x_coord + l], [y_coord - l, y_coord - l], c = 'r')
        plt.plot([x_coord - l, x_coord + l], [y_coord + l, y_coord + l], c = 'r')
        plt.plot([x_coord - l, x_coord - l], [y_coord - l, y_coord + l], c = 'r')
        plt.plot([x_coord + l, x_coord + l], [y_coord - l, y_coord + l], c = 'r')
        plt.text(x_coord, y_coord, str(box_number), c = 'r')
        
        box = data[y_coord - l:y_coord + l, x_coord - l:x_coord + l]
        print(f"Total signal count in box {num}: {sum(box.flatten())} ADU")
    
    view_fits(data, vmin = vmin, vmax = vmax, log = log)    
    
    num = 0
    for coords in coords_list:
        num += 1
        draw_box(coords[0], coords[1], side_length, num)
        

In [139]:
def make_box(coords, data, side_length: int = 10, vmin: int = 1, vmax: int = 65000, log: bool = True):
    """
    Make a box around a given coordinate in the format [x, y] in the FITS data array.
    Set the side length of the box in pixels using side_length.
    
    Use vmin and vmax to scale the image.
    Use log to swap between a linear and a logarithmic scale.
    
    Note: If the box isn't right, try swapping the x and y coords.
    """
    x = coords[0]
    y = coords[1]
    l = side_length//2
    
    print(f"Box of side length {side_length} centered around coordinates [{x}, {y}]")
    
    box = data[y-l:y+l, x-l:x+l]
    
    view_fits(box, vmin = vmin, vmax = vmax, log = log)
    plt.xticks(range(0, side_length+1, 5), [(i + x - l) for i in range(0, side_length+1, 5)])
    plt.yticks(range(0, side_length+1, 5), [(j + y - l) for j in range(0, side_length+1, 5)])
    
    print(f"Total signal count in box: {sum(box.flatten())} ADU")
    
    return box

In [ ]:
def make_histograms(box):
    """
    Makes 1D histograms of the total signal in x and the total signal in y.
    """
    stack_x = np.zeros(box.shape[1])
    stack_y = np.zeros(box.shape[0])
    for j in range(box.shape[0]):
        stack_x += box[j,:]
        stack_y += box[:,j]
        
    coords_range = range(box.shape[0])
    
    plt.bar(coords_range, stack_x, edgecolor='red'  , linewidth=1, fill=False, label = 'X', alpha = 0.5, width = 1)
    plt.bar(coords_range, stack_y, edgecolor='green', linewidth=1, fill=False, label = 'Y', alpha = 0.5, width = 1)
    
    plt.title('Signals in x and y')
    plt.xlabel('Box Coordinates [px]')
    plt.ylabel('Total Signal [ADU]')
    plt.legend()
    
    return stack_x, stack_y

### TESTING THE CODE ###

In [151]:
data = load_image('/Users/adi/Downloads/Ximea_BPfilter530_BlueLight_200ms.fits')[500:1700, 425:1650] #[900:1200, 900:950]

data[np.isnan(data)] = 0

In [172]:
view_fits(data, vmin = 10, vmax = 300, log = True)

In [173]:
view_boxes_on_image(data, [[500, 425], [500,550], [500, 675]], vmin = 10, vmax = 500, log = True, side_length = 45)

In [165]:
b34 = make_box([500, 425], data, side_length= 45, vmin= 10, vmax= 300)

In [155]:
plt.hist(b34.flatten(), label = '34', fill=False, edgecolor='r')
plt.hist(b36.flatten(), label = '36', fill=False, edgecolor='g')
plt.legend()

In [171]:
b36 = make_box([500, 675], data, side_length= 45, vmin= 10, vmax= 300)